## Build Environment

In [2]:
# Download the 1.4 sd model
from IPython.display import clear_output

from huggingface_hub import hf_hub_download
downloaded_model_path = hf_hub_download(
 repo_id="stabilityai/stable-diffusion-2",
 filename="768-v-ema.ckpt",
 use_auth_token=True
)

# Move the sd-v1-4.ckpt to the root of this directory as "model.ckpt"
actual_locations_of_model_blob = !readlink -f {downloaded_model_path}
!mv {actual_locations_of_model_blob[-1]} sd2.ckpt
clear_output()
print("✅ model.ckpt successfully downloaded")

✅ model.ckpt successfully downloaded


In [5]:
def get_format_ts():
    import time
    from datetime import datetime
    return datetime.fromtimestamp(int(time.time())).strftime('%Y%m%d-%H%M%S')

def get_img_urls(file_name):
    urls = []
    with open(file_name, "r") as f:
        lines = f.readlines()
        urls = [r.replace("\n", "") for r in lines]
    print(urls)
    return urls


def generate_training_images(urls, save_path):
    import os
    import requests
    from io import BytesIO
    from PIL import Image


    def image_grid(imgs, rows, cols):
     assert len(imgs) == rows*cols

     w, h = imgs[0].size
     grid = Image.new('RGB', size=(cols*w, rows*h))
     grid_w, grid_h = grid.size

     for i, img in enumerate(imgs):
      grid.paste(img, box=(i%cols*w, i//cols*h))
     return grid

    def download_image(url):
     try:
      response = requests.get(url)
     except:
      return None
     return Image.open(BytesIO(response.content)).convert("RGB")

    images = list(filter(None,[download_image(url) for url in urls]))
    save_path = "./training_images/{}".format(save_path)
    if not os.path.exists(save_path):
     os.mkdir(save_path)
    [image.save(f"{save_path}/{i}.png", format="png") for i, image in enumerate(images)]
    image_grid(images, 1, len(images))


def generate_reg_images(class_token, resume_model="v1-5.ckpt", self_generated_files_count=200):
    !rm -rf ./outputs
    !python scripts/stable_txt2img.py \
     --seed 10 \
     --ddim_eta 0.0 \
     --n_samples 1 \
     --n_iter {self_generated_files_count} \
     --scale 10.0 \
     --ddim_steps 50 \
     --ckpt resume_model/{resume_model} \
     --prompt {class_token}

    !mkdir -p regularization_images/{class_token}
    !mv outputs/txt2img-samples/*.png regularization_images/{class_token}

def train_concept(
    class_token, 
    prompt_token, 
    resume_model="v1-5.ckpt", 
    is_gen_training_images=False, 
    is_gen_reg_images=False, 
    reg_count=1000, 
    training_image_path=None,
    max_training_steps=2000):
    # write logs
    with open("./logs/training_logs.txt", "a+") as f:
        content = "ts: {}, from: {}, to: {}\n".format(get_format_ts(), resume_model.replace(".ckpt", ""), prompt_token)
        f.writelines(content)
        f.close()
    if is_gen_training_images:
        training_image_path = prompt_token + "_" + get_format_ts()
        urls = get_img_urls("./img_urls/{}.txt".format(prompt_token))
        generate_training_images(urls, training_image_path)
    else:
        assert training_image_path is not None
    
    if is_gen_reg_images:
        self_generated_files_count = reg_count
        generate_reg_images(class_token, resume_model, self_generated_files_count=self_generated_files_count)
    
    project_name = prompt_token
    
    # MAX STEPS
    # Match class_word to the category of the regularization images you chose above.
    class_word = class_token # typical uses are "man", "person", "woman"
    # This is the unique token you are incorporating into the stable diffusion model.
    token = prompt_token

    reg_data_root = "./regularization_images/" + class_token

    !rm -rf training_images/.ipynb_checkpoints
    !python "main.py" \
     --base configs/stable-diffusion/v1-finetune_unfrozen-mu.yaml \
     -t \
     --reg_data_root "{reg_data_root}" \
     -n "{project_name}" \
     --gpus 0, \
     --data_root "./training_images/{training_image_path}" \
     --max_training_steps {max_training_steps} \
     --class_word "{class_word}" \
     --token "{token}" \
     --no-test
#      --actual_resume "./resume_model/{resume_model}" \

In [4]:
%env USE_MEMORY_EFFICIENT_ATTENTION=1
# 定义新变量
def get_format_ts():
    import time
    from datetime import datetime
    return datetime.fromtimestamp(int(time.time())).strftime('%Y%m%d-%H%M%S')
class_token = "style"  # 这个变量是要训练的内容的类目，要用具有实际含义的词，如要训练一种特殊风格的椅子，就用chair, 训练特殊风格的人物，就用person, etc， 如果是一类风格的话，就用style
prompt_token = "test" # 这个变量要用模型当中没有出现过的词语
resume_model = "model.ckpt"  # 注意，这里是模型训练的启动点，意思是模型基于已有的哪个模型训练，在训练前问下良伟
# resume_model = "model.ckpt"

is_gen_training_images = True
is_gen_reg_images = True
reg_count = 4
training_image_path = None
max_training_steps=100
train_concept(class_token, prompt_token, resume_model, is_gen_training_images, is_gen_reg_images, reg_count, training_image_path,max_training_steps)

env: USE_MEMORY_EFFICIENT_ATTENTION=1
['https://i.imgur.com/XQxNKKl.png', 'https://i.imgur.com/V0URx9M.png', 'https://i.imgur.com/XpwebkP.png', 'https://i.imgur.com/5fzZAi3.png', 'https://i.imgur.com/zq4VxSD.png', 'https://i.imgur.com/3mQQhQT.png', 'https://i.imgur.com/CEqf6Cd.png', 'https://i.imgur.com/Snn6rV4.png', 'https://i.imgur.com/Wu9uKtC.png', 'https://i.imgur.com/6af50U3.png', 'https://i.imgur.com/OM4Zq40.png', 'https://i.imgur.com/QIJVOH8.png', 'https://i.imgur.com/75NDeZA.png', 'https://i.imgur.com/PkiWOe8.png', 'https://i.imgur.com/BDxVN0i.png', 'https://i.imgur.com/rPrKSgm.png', 'https://i.imgur.com/pWktxhc.png', 'https://i.imgur.com/lIDxPYC.png', 'https://i.imgur.com/vYrGj5i.png', 'https://i.imgur.com/zJb30eq.png']
Global seed set to 10
Loading model from resume_model/model.ckpt
Global Step: 470000
LatentDiffusion: Running in eps-prediction mode
Traceback (most recent call last):
  File "/home/pai/jupyter/fine_tune/exp/Dreambooth-Stable-Diffusion/scripts/stable_txt2img.py